### nudged training surface flux bias maps

Compare the effects of setting precipitation in the nudged run on time-mean surface turbulent flux biases

In [ ]:
import xarray as xr
import numpy as np
import intake
import cftime
from matplotlib import pyplot as plt
import matplotlib as mpl
from vcm.catalog import catalog
from vcm.fv3.metadata import standardize_fv3_diagnostics
import fv3viz as viz
import os
from dask.diagnostics import ProgressBar
import warnings

In [ ]:
mpl.rcParams.update({'font.size': 8})

In [ ]:
# paths and experiments

CASES = {
    'baseline': 'gs://vcm-ml-experiments/2021-04-13/baseline-physics-run-20160801.010000-start-rad-step-1800s',
    'nudging only': 'gs://vcm-ml-experiments/2021-04-13-n2f-c3072/3-hrly-ave-control-30-min-rad-timestep-shifted-start-tke-edmf',
    'nudging + prescribed rad. & precip': 'gs://vcm-ml-experiments/2021-04-13-n2f-c3072/3-hrly-ave-rad-precip-setting-30-min-rad-timestep-shifted-start-tke-edmf'
}

verification_physics_entry = '40day_c48_gfsphysics_15min_may2020'
output_dir = 'figures'

In [ ]:
# load run datasets

drop_vars = [
    'column_heating_nudge',
    'column_moistening_nudge',
    'column_mass_tendency_nudge',
    'net_moistening_due_to_nudging',
    'net_heating_due_to_nudging',
    'net_mass_tendency_due_to_nudging',
    'column_integrated_dQu',
    'column_integrated_dQv',
    'net_heating',
    'net_moistening',
    'total_precip',
    'total_precipitation_rate',
    'USWRFsfc_from_RRTMG',
    'DSWRFsfc_from_RRTMG',
    'DLWRFsfc_from_RRTMG'
]

run_datasets = []
for case, url in CASES.items():
    print(f"{case}: {url}")
    dycore = (
        standardize_fv3_diagnostics(intake.open_zarr(os.path.join(url, 'atmos_dt_atmos.zarr')).to_dask())
        .resample(time="1H", label="right").nearest()
    )
    physics = (
        standardize_fv3_diagnostics(intake.open_zarr(os.path.join(url, 'sfc_dt_atmos.zarr')).to_dask())
        .resample(time="1H", label="right").nearest()
    )
    diags = (
        standardize_fv3_diagnostics(intake.open_zarr(os.path.join(url, 'diags.zarr')).to_dask())
        .resample(time="1H", label="right").nearest()
    )
    merged = xr.merge([dycore, physics, diags]).drop_vars(names=drop_vars, errors='ignore')
    run_datasets.append(merged.assign_coords(cases=[case]))
run_dataset = xr.concat(run_datasets, dim='cases', join='inner')

run_dataset = run_dataset.loc[{'time': run_dataset.time > cftime.DatetimeJulian(2016, 8, 5, 1, 0, 0, 0)}]

In [ ]:
# load verification dataset and subsample to hourly
verification_dataset = (
    standardize_fv3_diagnostics(catalog[verification_physics_entry].to_dask())
    .resample(time="1H", label="right").nearest()
)

In [ ]:
# use common times for both runs and verif
common_time = verification_dataset.time.loc[verification_dataset.time > run_dataset.time[0]]
run_dataset = run_dataset.sel(time=common_time)
verification_dataset = verification_dataset.sel(time=common_time)

In [ ]:
def add_total_downward_surface_radiative_flux(ds: xr.Dataset) -> xr.Dataset:
    DRFsfc = ds['DSWRFsfc'] + ds['DLWRFsfc']
    DRFsfc = DRFsfc.assign_attrs({
        'long_name': 'total downward radiative flux at surface',
        'units': 'W/m^2'
    })
    ds['DRFsfc'] = DRFsfc
    return ds

In [ ]:
run_dataset = add_total_downward_surface_radiative_flux(run_dataset)
verification_dataset = add_total_downward_surface_radiative_flux(verification_dataset)

In [ ]:
TIME_MEANS = [
    'DLWRFsfc',
    'DSWRFsfc',
    'LHTFLsfc',
    'SHTFLsfc',
    'DRFsfc',
]

def time_mean_variable(da: xr.DataArray) -> xr.DataArray:
    time_mean_da = da.mean(dim=['time'])
    time_mean_da.attrs = da.attrs
    return time_mean_da

def add_time_means(ds: xr.Dataset) -> xr.Dataset:
    for name in TIME_MEANS:
        try:
            ds = ds.assign({f'{name}_time_mean': time_mean_variable(ds[name])})
        except KeyError:
            print(f"Variable could not be computed: {name}")
    return ds

In [ ]:
run_dataset = add_time_means(run_dataset)
verification_dataset = add_time_means(verification_dataset)

In [ ]:
# open grid

grid_c48 = standardize_fv3_diagnostics(catalog["grid/c48"].to_dask())
ls_mask = standardize_fv3_diagnostics(catalog["landseamask/c48"].to_dask())
grid_c48 = xr.merge([grid_c48, ls_mask])

MAPPABLE_VAR_KWARGS = {
    "coord_x_center": "x",
    "coord_y_center": "y",
    "coord_x_outer": "x_interface",
    "coord_y_outer": "y_interface",
    "coord_vars": {
        "lonb": ["y_interface", "x_interface", "tile"],
        "latb": ["y_interface", "x_interface", "tile"],
        "lon": ["y", "x", "tile"],
        "lat": ["y", "x", "tile"],
    },
}

In [ ]:
def prognostic_time_mean_maps(da, row=None, title=None, plot_cube_kwargs=None, bias=None, fig_size=[14, 4]):
    plot_cube_kwargs = plot_cube_kwargs or {}
    fig, axes, _, _, facetgrid = viz.plot_cube(da, col='cases', row=row, **plot_cube_kwargs)
    facetgrid.set_titles(template='{value}')
    if bias:
        if row:
            for i in range(da.sizes[row]):
                row_axes = axes[i]
                for j, case in enumerate(da.cases.values):
                    ax = row_axes[j]
                    if i == 0:
                        ax.set_title(
                            f"{title if title else case}\n"
                            f"{bias[0]}: {bias[1][j, i]:3.1f}"
                        )
                    else:
                        ax.set_title(
                            f"{bias[0]}: {bias[1][j, i]:3.1f}"
                        )
                        
        else:
            for i, case in enumerate(da.cases.values):
                ax = axes.flatten()[i]
                ax.set_title(
                    f"{title if title else case}\n"
                    f"{bias[0]}: {bias[1][i]:3.1f}"
                )
    fig.set_size_inches(fig_size)
    return fig
                              
    
def _var_rms(da, area):
    weights = area/area.mean()
    return np.sqrt(((da**2)*weights).mean(dim=['x', 'y', 'tile']))

def _var_mean(da, area):
    weights = area/area.mean()
    return (weights*da).mean(dim=['x', 'y', 'tile'])

def _var_land_mean(da, area, mask):
    area = area.where(mask == 1.0)
    return (da*area).sum(dim=['x', 'y', 'tile'])/(area.sum(dim=['x', 'y', 'tile']))

In [ ]:
var = 'DLWRFsfc_time_mean'
diff_ds = (
    viz.mappable_var(
        xr.merge([grid_c48, (verification_dataset - run_dataset).sel({'cases': ['nudging only']})], compat='override'),
        var,
        **MAPPABLE_VAR_KWARGS
    )
)
with ProgressBar():
    mean_ = _var_mean(diff_ds[var], grid_c48['area']).load()
    
print(mean_.item())

In [ ]:
var = 'DSWRFsfc_time_mean'
diff_ds = (
    viz.mappable_var(
        xr.merge([grid_c48, (verification_dataset - run_dataset).sel({'cases': ['nudging only']})], compat='override'),
        var,
        **MAPPABLE_VAR_KWARGS
    )
)
with ProgressBar():
    mean_ = _var_mean(diff_ds[var], grid_c48['area']).load()

print(mean_.item())

In [ ]:
var = 'DRFsfc_time_mean'
diff_ds = (
    viz.mappable_var(
        xr.merge([grid_c48, (verification_dataset - run_dataset).sel({'cases': ['nudging only']})], compat='override'),
        var,
        **MAPPABLE_VAR_KWARGS
    )
)
diff_ds[var] = diff_ds[var].assign_attrs({
    'long_name': 'fine - coarse bias',
    'units': '$W/m^2$'
})
with ProgressBar():
    diff_ds = diff_ds.load()
    mean_ = _var_mean(diff_ds[var], grid_c48['area']).load()
    
print(mean_.item())

In [ ]:
prognostic_time_mean_maps(
    diff_ds,
    bias=('mean', mean_.values),
    fig_size=[3.8, 2.5],
    title='Surface downwelling radiation bias'
)
plt.savefig(f"{output_dir}/Figure_A2_total_downwelling_bias_time_mean.eps", bbox_inches='tight')

In [ ]:
varnames = ['SHTFLsfc_time_mean', 'LHTFLsfc_time_mean']
combined_name = 'turbulent_fluxes'
run_da = run_dataset[varnames].to_array(dim='varnames', name=combined_name)
verification_da = verification_dataset[varnames].to_array(dim='varnames', name=combined_name)
diff_ds = (
    viz.mappable_var(
        xr.merge(
            [(run_da - verification_da).sel({'cases': ['nudging only', 'nudging + prescribed rad. & precip']}), grid_c48]
        ).assign_coords({'varnames': ['SHF', 'LHF']}),
        combined_name,
        **MAPPABLE_VAR_KWARGS
    )
)
diff_ds[combined_name] = diff_ds[combined_name].assign_attrs({'long_name': 'coarse - fine bias', 'units': '$W/m^2$'})
with ProgressBar():
    diff_ds = diff_ds.load()
    land_mean_ = _var_land_mean(diff_ds[combined_name], grid_c48['area'], grid_c48['land_sea_mask']).load()
    
print(land_mean_.values)

In [ ]:
fig = prognostic_time_mean_maps(
    diff_ds,
    row='varnames',
    bias=('land mean', land_mean_.values),
    plot_cube_kwargs=dict(vmin=-50, vmax=50),
    fig_size=[7.6, 5]
)
fig.savefig(f"{output_dir}/Figure_A4_turbulent_flux_bias_time_mean.eps", bbox_inches='tight')

In [ ]:
# this converts matplotlib eps files to a more manageable size

!epstopdf figures/Figure_A2_total_downwelling_bias_time_mean.eps
!pdftops -eps figures/Figure_A2_total_downwelling_bias_time_mean.pdf
!rm figures/Figure_A2_total_downwelling_bias_time_mean.pdf

!epstopdf figures/Figure_A4_turbulent_flux_bias_time_mean.eps
!pdftops -eps figures/Figure_A4_turbulent_flux_bias_time_mean.pdf
!rm figures/Figure_A4_turbulent_flux_bias_time_mean.pdf